In [1]:
 import mysql.connector 

In [2]:
#create a list of all airline user_ids for later on
airlines= [56377143, 106062176,18332190, 22536055,124476322,26223583,2182373406,38676903,1542862735,253340062,218730857,
          45621423,20626359]

In [3]:
db = mysql.connector.connect(host="127.0.0.1", user="root", password="",database="dbl")
cursor=db.cursor() #initialize mysql to be able to query and edit the SQL tables.

In [4]:
cursor.execute("SELECT * FROM `tweets` ORDER BY timestamp DESC")
total=cursor.fetchall() #store all tweets from new to old in the variable 'total'

Tables: <br>
tweets: (id, text, in_reply_to, timestamp, user_mentions, user_id, user_verified, user_followers_count, user_tweets_count) <br>
conversations: (id, Tstart, Tend, Airline, Length)<br>
part_of: (tID, cID)

In [5]:
def create_tables():
    """This function will create a new table in the MySQL environment called conversations with the following attributes:
    conversation ID, conversation start time, conversation end time, has an airline participated, length of conversations,
    and the response time. Furthermore, also create a table that links tweets to conversations they are a part of."""
    
    cursor.execute("""CREATE TABLE `dbl`.`conversations`(id MEDIUMINT NOT NULL, Tstart BIGINT NOT NULL, Tend BIGINT NOT NULL,
    Airline VARCHAR(50) NOT NULL DEFAULT '0',Length SMALLINT NOT NULL, PRIMARY KEY (id))""")
    
    cursor.execute("""CREATE TABLE `dbl`.`part_of`(tID BIGINT NOT NULL, cID MEDIUMINT NOT NULL, PRIMARY KEY(tID, cID), 
    FOREIGN KEY (tID) REFERENCES `tweets`(id), FOREIGN KEY (cID) REFERENCES `conversations`(id))""")
    return 'done'

def drop_tables():
    """Drop the table called conversations and the one called part_of"""
    cursor.execute("DROP TABLE `part_of`")
    cursor.execute("DROP TABLE `conversations`")

    return 'dropped'

def clear_tables():
    """Clears all values from the tables part_off and conversations"""
    cursor.execute("DELETE FROM `part_of`")
    cursor.execute("DELETE FROM`conversations`")
    return db.commit()

In [18]:
create_tables() #here we create the extra tables that we need to store the information about our conversations.

'done'

In [30]:
def fill_tables (tweets: list) -> None:
    """This function will insert into the tables 'part_of' and 'conversations' the correct information given an input
        SQL table tweets.
        """
    
    #store what conversations a certain tweet_id will have to become a member of,
    member_of = {} #because a reply to it is in it, in this dict.

    #increment this whenever you create a new conversation, to ensure there are no duplicate ids, we start with cID=1
    conv_id = 1
    
    n = len(tweets) #the number of rows in the tweets table
    run = 0 #keep track of how many runs have been done.
    for t in tweets:
        tweet = t[0]
        reply_id = t[2]
        tstamp = t[3]
        user_id = t[5]
        if tweet in member_of: #If we have found replies to this tweet
            convs = member_of.pop(tweet)
            for conv in convs:
                cursor.execute(f"INSERT INTO `part_of`(tID,cID) VALUES ({tweet},{conv})")
                
                cursor.execute(f"SELECT * FROM `conversations` WHERE id={conv}") 
                
                conv_info=cursor.fetchall() #store the information about the current conversation in conv_info
                
                conv_airline = conv_info[0][3] #the airlines that are participating in this conversation
                length = 1+conv_info[0][4] #the current length of this conversations
                conv_airline_lst = list(map(int,conv_airline.split(","))) #turn the string into a list of integers
                
                if user_id in airlines:    #if the user is an airline
                    
                    if conv_airline_lst == [0]: #if no airline has participated in this conversation.
                        cursor.execute(f"""UPDATE `conversations` SET Tstart={tstamp}, Airline = {user_id}, length = {length}
                        WHERE id={conv}""") #update the table and keep track of what airline has participated
                    
                    elif user_id not in conv_airline_lst:#if this airline has not yet participated, but another one has
                        conv_airline_new =  conv_airline_lst+ [user_id] #store the airlines already participating
                                                                #and the new one in a list
                        for line in range(len(conv_airline_new)): #Create list of airlines that have participated.
                            
                            if line == 0: #if its the first airline, no space needed
                                str_airline=str(conv_airline_new[line])
                                
                            else:
                                str_airline = str_airline+ "," +str(conv_airline_new[line])
                                
                        cursor.execute(f"""UPDATE `conversations` SET Tstart={tstamp}, Airline = '{str_airline}',
                        length = {length} WHERE id={conv}""")
                        
                    else: #if the airline has already participated
                        cursor.execute(f"UPDATE `conversations` SET Tstart={tstamp}, length = {length} WHERE id={conv}")

                else: #If the user is not an airline
                    cursor.execute(f"UPDATE `conversations` SET Tstart={tstamp}, length = {length} WHERE id={conv}")
            
        else: #this is the last tweet of a conversation (it has no replies)
            convs=[conv_id] #used for later if statement.
            
            if user_id in airlines: #if the user is an airline
                cursor.execute(f"""INSERT INTO `conversations`(id,Tstart, Tend, Airline, length)
                VALUES ({conv_id}, {tstamp}, {tstamp}, {user_id}, 1)""") #create conversation in table
            
            else: #The tweet is not from an airline
                cursor.execute(f"""INSERT INTO `conversations`(id,Tstart, Tend, length) 
                VALUES ({conv_id}, {tstamp}, {tstamp}, 1)""") #create conversation in table
            
            cursor.execute(f"INSERT INTO `part_of`(tID,cID) VALUES ({tweet},{conv_id})") #update the part_of table
            conv_id += 1 #increment the conversation id to ensure no duplicate conversation ids occur
        
        if reply_id != 0: #if this tweet replies to someone
            
            if reply_id in member_of:   #if the tweet this replies to already has another reply, add the new conversations to 
                                        #dict and keep old one
                member_of[reply_id] = member_of[reply_id] + convs
                
            else:
                member_of[reply_id] = convs #If there is not yet another reply to this tweet, store what conversation it
                                            #is now a part of.
        if run% round(n/10)==0: #run is a multiple of n/10 rounded to an integer
            db.commit() #commit the database to ensure it does not run into an error
        run += 1
    return None #db.commit()

In [31]:
FT_tot = fill_tables(total)